# Simple seq2seq example using LSTMs with TensorFlow

Based on Matvey Ezhov's [materials](https://github.com/ematvey/tensorflow-seq2seq-tutorials) under the [MIT License](https://opensource.org/licenses/MIT)


This exercise helps you implement a basic seq2seq model comprised of a forward-only encoder - decoder, based on the architecture described in [Sutskever, Vinyals and Le (2014)](https://arxiv.org/abs/1409.3215).

Architecture diagram from their paper:
![seq2seq architecutre](seq2seq.png)
Rectangles are encoder and decoder's recurrent layers. The sequence `[A, B, C]` is the **encoder input**.  The encoder's hidden state changes while reading the sequence.  After input sequence ends (indicated by `<EOS>`), the encoder passes its final state to decoder, which receives `[<EOS>, W, X, Y, Z]` as the **decoder input** and is trained to the **decoder output** `[W, X, Y, Z, <EOS>]`. `<EOS>` token is a special word in vocabulary that signals to decoder the beginning of translation.

## Vocabulary

Seq2seq maps one sequence onto another sequence. Both sequences consist of integers from a fixed range. In language tasks, integers usually correspond to words: we first construct a vocabulary by assigning to every word in our corpus a serial integer. First few integers are reserved for special tokens. We'll call the upper bound on vocabulary a `vocabulary size`.

A lot of processing goes into making text into numbers.  To simplify things, this notebook will focus instead on trying to learn a sequence of numbers.

In [ ]:
# For instance, here are four separate sequences, all of different lengths
x = [[5, 7, 8], [6, 3], [3], [1]]
for i, seq in enumerate(x):
    print("Sequence {0}: {1}".format(i,seq))

While manipulating such variable-length lists are convenient to humans, RNNs prefer a different layout called **time-major** where sequences are found in the columns of a matrix of size `[max_time, batch_size].`  Sequences shorter than the longest one are padded with zeros at the end. 

In [ ]:
import helpers  # uses helper file - check out at you leisure
xt, xlen = helpers.batch(x)

In [ ]:
for i, batch in enumerate(xt):
    print("Time {0}: {1}".format(i,batch))

In [ ]:
print("The length of each sequence in each column:")
print(xlen)

# Building a model

## Simple seq2seq

The encoder (an LSTM) starts with an initial state and runs through the input sequence until it reaches its `final_state`.

The decoder uses the encoder's `final_state` as its `initial_state`.

Therefore there are **encoder inputs** and **decoder inputs** and **decoder targets.** The decoder's outputs are mapped onto the output space using a `[hidden_units x output_vocab_size]` size matrix. 

Both the encoder and decoder are trying to learn sequences.  Tensorflow's [LSTMCells](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) will be used to model the sequences.

#### Imports 

In [ ]:
# imports
# import numpy, tensorflow, and also import helpers (like we did above in this worksheet)
# remember conventions for importing numpy and tensorflow



In [ ]:
# what version of tensorflow are you running?
tf.__version__

### Model inputs and outputs 

First critical thing to decide: vocabulary size.

Dynamic RNN models can be adapted to different batch sizes and sequence lengths without retraining, but changing vocabulary size requires retraining the model.

In [ ]:
PAD = 0 # if a sequence is short, what should it be padded with to get it up to some desired length?
EOS = 1 # what signifies the end of a sequence

vocab_size = ??  # how many different single digit numbers could we see? 0,1,2,3,5,6,7,8,9

## Define the tensors
Remember to use tf.placeholder.  Generally the shape will be `(max_time, batch_size)` but the values of max_time and batch size may change so we want to initialize them with [unknown shapes.](https://www.tensorflow.org/api_docs/python/tf/TensorShape) For datatype use 32 bit integers.

In [ ]:
encoder_inputs = ??
decoder_inputs = ??
decoder_targets = ??

### Embeddings

Let's give the `encoder` and `decoder` more meaningful inputs than just the sequence numbers. We can embed context (i.e. if 4 then 5 happens then 1 is probably next) into the inputs.  This process of embedding will take vectors that are sparse (filled with many zeros) and make then dense, which is required for the encoder and decoder RNNs.  Specifics of working with embeddings are nicely described in [official tutorial on embeddings](https://www.tensorflow.org/tutorials/word2vec/).

First initialize the embedding matrix. Initializations are random. We rely on our end-to-end training to learn vector representations for the embeddings for the sequences of numbers jointly with encoder and decoder.

#### Initialize the embeddings as a tensorflow variable using a [random uniform](https://www.tensorflow.org/api_docs/python/tf/random_uniform) distribution between -1.0 and 1.0, datatype float.  Its shape should be (vocab_size, input_embedding_size).

In [ ]:
input_embedding_size = 5 # how many columns we think we'll need to capture the relationship between the numbers
embeddings = ?? # you'll need a tensorflow variable initialized according to the instructions above

## Define the operations
Start with the embeddings.  In this case the seq2seq network will just be learning a sequence of digits, so the encoder inputs and the decoders inputs (and embeddings) are the same.

In [ ]:
encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)
decoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, decoder_inputs)

### Encoder

The encoder is a [dynamic rnn](https://www.tensorflow.org/versions/master/api_docs/python/tf/nn/dynamic_rnn) comprised of [LSTMCells.](https://www.tensorflow.org/versions/master/api_docs/python/tf/contrib/rnn/LSTMCell) 

In [ ]:
encoder_hidden_units = 15 # how many units in the LSTM cell?

encoder_cell = ?? #use the LSTMCell and pass in the number of units

_, encoder_final_state = tf.nn.dynamic_rnn(?? #the cell you just defined, 
                                           ?? #the embedded encoder inputs,
                                           dtype=tf.float32,
                                           time_major=True)

The `encoder_final_state` is also called "thought vector".  In this simple seq2seq model this is the only point where Encoder passes information to Decoder.  Backpropagation through time (BPTT) algorithm will tune the model to pass enough information throught the thought vector for correct sequence output decoding.

In [ ]:
encoder_final_state

### Decoder

The decoder is also a [dynamic rnn](https://www.tensorflow.org/versions/master/api_docs/python/tf/nn/dynamic_rnn) comprised of [LSTMCells.](https://www.tensorflow.org/versions/master/api_docs/python/tf/contrib/rnn/LSTMCell) 
Give it the same number of hidden units as the encoder.

In [ ]:
decoder_hidden_units = ?? #same number of units as the encoder

decoder_cell = ?? #just like the encoder

decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(?? #cell you just defined,
                                                         ?? #embedded decoder inputs,
                                                         initial_state=encoder_final_state,
                                                         dtype=tf.float32, 
                                                         time_major=True,
                                                         scope="plain_decoder",)

At this point `decoder_cell` output is a `hidden_units` sized vector at every timestep. However, for training and prediction we need logits of size `vocab_size`. Reasonable thing would be to put linear layer (fully-connected layer without activation function) on top of LSTM output to get non-normalized logits. This layer is called projection layer by convention.

In [ ]:
decoder_logits = tf.contrib.layers.linear(decoder_outputs, vocab_size)
decoder_prediction = tf.argmax(decoder_logits, 2)

In [ ]:
decoder_logits

RNN outputs tensor of shape `[max_time, batch_size, hidden_units]` which projection layer maps onto `[max_time, batch_size, vocab_size]`. `vocab_size` part of the shape is static, while `max_time` and `batch_size` are dynamic.

## Define the Loss and Optimizer
For the loss get the [mean](https://www.tensorflow.org/versions/master/api_docs/python/tf/reduce_mean) stepwise_cross_entropy.  For the optimizer, think about using [Adam.](https://www.tensorflow.org/versions/master/api_docs/python/tf/train/AdamOptimizer)

In [ ]:
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(decoder_targets, depth=vocab_size, dtype=tf.float32),
    logits=decoder_logits,
)

loss = ?? # how to define the loss
train_op = ?? # how to define the optimizer

### Make an initialize operation to initialize the variables

In [ ]:
# Initialize the variables (i.e. assign their default value)
init = ?? 

## Training on a toy data set

We will teach our model to memorize and reproduce input sequence. Sequences will be random, with varying length.

Since random sequences do not contain any structure, model will not be able to exploit any patterns in data. It will simply encode sequence in a thought vector, then decode from it.

In [ ]:
batch_size = 4

batches = helpers.random_sequences(length_from=3, length_to=8,
                                   vocab_lower=2, vocab_upper=10,
                                   batch_size=batch_size)
print('one batch:')
for seq in next(batches):
    print(seq)

In [ ]:
# from the batch it determines the encoder_input, the decoder_input, and the decoder_target
def next_feed(prnt=False):
    batch = next(batches)
    if prnt:
        print("The batch is:")
        for line in batch:
            print(line)
    encoder_inputs_, _ = helpers.batch(batch)
    decoder_targets_, _ = helpers.batch(
        [(sequence) + [EOS] for sequence in batch]
    )
    decoder_inputs_, _ = helpers.batch(
        [[EOS] + (sequence) for sequence in batch]
    )
    return {
        encoder_inputs: encoder_inputs_,
        decoder_inputs: decoder_inputs_,
        decoder_targets: decoder_targets_,
    }

In [ ]:
test=next_feed(prnt=True)

In [ ]:
print(test[encoder_inputs])

In [ ]:
print(test[decoder_inputs])

In [ ]:
print(test[decoder_targets])

## Start the session and train the model

In [ ]:
batch_size = 50

batches = helpers.random_sequences(length_from=3, length_to=8,
                                   vocab_lower=2, vocab_upper=10,
                                   batch_size=batch_size)

loss_track = []

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess ?? #how to run the initializer?

    max_batches = 2501
    print_interval = 500

    for batch in range(max_batches):
        fd = next_feed()
        _, l = sess.run([train_op, loss], ?? #need to pass in feed dictionary)
        loss_track.append(l)

        if batch == 0 or batch % print_interval == 0:
            print('batch {}'.format(batch))
            print('*'*40)
            print('  minibatch loss: {0:0.3f}'.format(sess.run(loss, fd)))
            predict_ = sess.run(decoder_prediction, fd)
            for i, (inp, pred) in enumerate(zip(fd[encoder_inputs].T, predict_.T)):
                print('  sample {}:'.format(i + 1))
                print('    input     > {}'.format(inp))
                print('    predicted > {}'.format(pred))
                if i >= 2:
                    break
            print()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(loss_track)
plt.ylabel('loss')
plt.xlabel('epoch')
print('loss {:.4f} after {} examples (batch_size={})'.format(loss_track[-1], len(loss_track)*batch_size, batch_size))

Something is definitely getting learned!

### Next steps:
Try changing the number of hidden units or the embedding size and see what happens to your results.